In [ ]:
from IPython.display import display, HTML

<div>
<img src="https://webassets.unt.edu/assets/branding/unt-stacked-logo.svg" alt="UNT | University of North Texas" class="desktop-logo" width="300" height="500">
<img src="media/Back-to-the-future-logo.svg" alt="BackToTheFuture" class="desktop-logo" width="600" height="1000">
</div>


<div style="display: flex; justify-content: space-around; padding: 20px 40px 20px 20px; background-color: #f4f4f9; border-radius: 10px;">
  <!-- Team Member 1 -->
  <div style="text-align: center; background-color: #ffffff; border-radius: 10px; padding: 20px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); width: 150px;">
    <img src="https://via.placeholder.com/100" style="border-radius: 50%; width: 100px; height: 100px;">
    <h3 style="font-family: Arial, sans-serif; color: #333;">Sonali Sabnam</h3>
    <p style="font-family: Arial, sans-serif; color: #666;">Da Boss</p>
  </div>
  
  <!-- Team Member 2 -->
  <div style="text-align: center; background-color: #ffffff; border-radius: 10px; padding: 20px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); width: 150px;">
    <img src="https://via.placeholder.com/100" style="border-radius: 50%; width: 100px; height: 100px;">
    <h3 style="font-family: Arial, sans-serif; color: #333;">Sonam Pohuja</h3>
    <p style="font-family: Arial, sans-serif; color: #666;">Da Other Boss</p>
  </div>

  <!-- Team Member 3 -->
  <div style="text-align: center; background-color: #ffffff; border-radius: 10px; padding: 20px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); width: 150px;">
    <img src="https://via.placeholder.com/100" style="border-radius: 50%; width: 100px; height: 100px;">
    <h3 style="font-family: Arial, sans-serif; color: #333;">Luis Garcia Fuentes</h3>
    <p style="font-family: Arial, sans-serif; color: #666;">Da Cool Guy</p>
  </div>

  <!-- Team Member 4 -->
  <div style="text-align: center; background-color: #ffffff; border-radius: 10px; padding: 20px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); width: 150px;">
    <img src="https://via.placeholder.com/100" style="border-radius: 50%; width: 100px; height: 100px;">
    <h3 style="font-family: Arial, sans-serif; color: #333;">Young Yu</h3>
    <p style="font-family: Arial, sans-serif; color: #666;">Da Janitor</p>
  </div>
</div>

# Back to the Futures: The AI Stock Predictor
Back to the Futures is a project where we try to do the impossible—predict the stock market with AI! <br>
Because if there’s one thing the stock market loves, it’s being perfectly predictable.  <br>
(spoiler: it’s not). <br>
- Our goal? To use machine learning to turn volatility into victory.
- Will it work? Well, let’s just say our fallback plan involves a lot of ramen.

In [1]:
# Add your own imports here
import pandas as pd



# Do Not Edit Below the lines
#-------------------------------------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------------------------
from utilities.api import *
from utilities.common import *
from utilities.data_utils import *
from utilities.classify import *
from utilities.db_tools import conn
import tensorflow.config as tfc # type: ignore
from transformers import pipeline # type: ignore
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
'''
We're just setting the GPU here, if you don't have one... no worries, I don't either.
It will fall back to your CPU :)
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
# device = set_device(use_gpu=True)
# print(f"Using {device}")
if tfc.list_physical_devices('GPU'):
    device = 0  # Use the first GPU
else:
    device = -1 

In [3]:
'''
Load your environmental variables. Define a few others.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
api_key = os.getenv('NEWS_API_KEY')
base_dir = os.getenv('ROOT_DIR')
project_name = 'adta_5410'
version = 'v1'

In [4]:
'''
Create a project directory and subfolders
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
project_path = create_project(project_name, version)

# Load the project-specific environment variables from .env_temp
from dotenv import load_dotenv

load_dotenv(os.path.join(project_path, ".env_temp"))


Set environment variable: CHARTS_DIR=adta_5410_v1_2024-10-15/charts
Set environment variable: DATA_DIR=adta_5410_v1_2024-10-15/data
Set environment variable: MODELS_DIR=adta_5410_v1_2024-10-15/models
Set environment variable: PIPELINE_DIR=adta_5410_v1_2024-10-15/pipeline
Set environment variable: LOGS_DIR=adta_5410_v1_2024-10-15/logs
.env_temp file created at: adta_5410_v1_2024-10-15/.env_temp
Set environment variable: PROJECT_DIR=adta_5410_v1_2024-10-15


True

In [5]:
'''
We need to pick a company in the S&P 500 to train our model on.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
eval_start = '2024-09-16'
eval_stop = '2024-10-10'
company = 'Tesla'
ticker = 'TSLA'

In [6]:
'''
This function gets the news. There are limitations. Check out the README.md. 
If you are getting a "Error fetching data: 426 Client Error: Upgrade Required for url",
It is because of the developer license limitations. Adjust your eval_start in the cell above.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
articles = get_news(api_key, company, eval_start, eval_stop)
# for article in articles:
#     print(f"Title: {article['title']}")
#     print(f"Published: {article['publishedAt']}")
#     print(f"Content: {article['content']}")

In [ ]:
'''
This is where we define our pre-trained model. The one below is just a place holder.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
# Need someone to train this model
classifier = pipeline("zero-shot-classification", 
                      model="facebook/bart-large-mnli",
                      device=device
                      )

In [ ]:
'''
This is where we define our pre-trained model. The one below is just a place holder.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
# Need someone to train this model
sentiment_classifier = pipeline("sentiment-analysis", 
                                model="distilbert-base-uncased-finetuned-sst-2-english",
                                device=device
                                )

In [9]:
'''
This is where we call the classifier model to filter out the financial news from the garbage.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
# articles.drop(columns='Unnamed: 0')
filtered_articles = [
        article for article in articles if is_financial_article(article, classifier)
    ]
print(f"Filtered {len(filtered_articles)} financial articles.")

Filtered 28 financial articles.


In [10]:
'''
Put the article data in a dataframe
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
data = []
for article in articles:
    sentiment = article_sentiment(article, sentiment_classifier)
    data.append({
        'date': article.get('publishedAt', ''),
        'title': article['title'],
        'content': article.get('content', ''),
        'sentiment_score': sentiment    
    })
articles = pd.DataFrame(data)

In [11]:
'''
Here we use our trusty yfinance module to get stocks. The variables are already defined above.
Notice we are pulling the same date ranges as the news articles.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
stocks = get_stocks(ticker, eval_start, eval_stop)

[*********************100%***********************]  1 of 1 completed


In [12]:
'''
We are going to join the two dataframes on "date" column. In order to do that,
we need the datetime formats to match. Basically, it converts the column names in
the dataframes to match, reformats the datetime, then trims off the time component.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
df_stocks = format_date(stocks)
df_articles = format_date(articles)

In [13]:
'''
Saving ourselves some heart ache later on.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
df_stocks = format_column_names(df_stocks)
df_articles = format_column_names(df_articles)

In [14]:
'''
Okay, here we merge everything on date. The fillna is there just in case one of the api's 
returns a null.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
merged_data = pd.merge(df_stocks, df_articles, on='date', how='outer').fillna(0)

In [15]:
'''
Here we drop some columns we don't plan on using. Regression models aren't fans of non-numerical data.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
merged_data.drop(columns=['title','content'], axis=1, inplace=True)
merged_data['sentiment_count'] = 1

In [ ]:
'''
We need to aggregate the sentiment scores for the weekends and apply them to the following monday.
Weekend news can impact opening trades on Monday. TODO: Work in Progress Weekends and Holidays
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
# weekend_df = handle_weekends(merged_data)

In [16]:
'''
This is defining a dictionary structure that I'll pass to a function. On some days, you'll get several news articles.
Our prediction is based on the aggregation of all financial news per day.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
agg_columns = {
    'open': 'mean',
    'high': 'mean',
    'low': 'mean',
    'close': 'mean',
    'adj_close': 'mean',
    'volume': 'mean',  
    'sentiment_score': 'mean', 
    'sentiment_count': 'count' 
}

In [17]:
'''
We pass the dict along with the dataframe to the function. Your output should be 1 row per day.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
df_aggregated = aggregate_column(merged_data, agg_columns)

In [18]:
'''
save files as needed
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
filename = f'{company}'
filepath = os.getenv('DATA_DIR')
df_sentiment_saved = save_data(df_articles, filename+'_articles', filepath, xlsx=False)
df_stock_saved = save_data(df_stocks, filename+'_stocks', filepath, xlsx=False)
df_final_saved = save_data(df_aggregated, filename+'_merged', filepath, xlsx=False)

File saved successfully at adta_5410_v1_2024-10-15/data/Tesla_articles.csv
File saved successfully at adta_5410_v1_2024-10-15/data/Tesla_stocks.csv
File saved successfully at adta_5410_v1_2024-10-15/data/Tesla_merged.csv


In [19]:
'''
Save the DataFranme into the database for later processign.
'''

#-------------------------------------------------------------------------------------------------------------------------------------------------------------
df_to_postgres(df_aggregated, 'training_data', conn)

conn.close()

Table 'training_data' is ready.
Inserted batch 1 successfully
